In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

# Define the path to your dataset directory
dataset_dir = '/content/drive/MyDrive/TrashNet'

# Check if the directory exists
if os.path.exists(dataset_dir):
    print("Dataset directory exists.")
else:
    print("Dataset directory does not exist.")

Dataset directory exists.


In [ ]:
import numpy as np # For numerical computations
import pandas as pd # For dataframe operations

# Importing Matplotlib and seaborn for Data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# For mathematical calculations
import math

# All tensorflow utilities for creating, training and working with a CNN.
import tensorflow as tf # (version 2.8.4)
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout,BatchNormalization
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
# For evaluation matrices for comparative analysis
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, roc_curve, auc, classification_report, confusion_matrix


In [4]:
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Define the path to the dataset
dataset_dir = '/content/drive/MyDrive/TrashNet'

# Categories in the dataset
categories = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']

# Load images and labels
data = []
labels = []

for category in categories:
    path = os.path.join(dataset_dir, category)
    class_num = categories.index(category)
    if not os.path.isdir(path):
        print(f"Directory {path} not found.")
        continue
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        try:
            img_array = cv2.imread(img_path)
            if img_array is not None:
                img_array = cv2.resize(img_array, (128, 128))
                data.append(img_array)
                labels.append(class_num)
            else:
                print(f"Failed to load image {img_path}")
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")

# Convert to numpy arrays
data = np.array(data)
labels = np.array(labels)

# Check if data is loaded correctly
print(f"Loaded {len(data)} images.")

# Ensure there is data to split
if len(data) == 0:
    raise ValueError("No images loaded. Check dataset path and structure.")

# Normalize data
data = data / 255.0

# One-hot encode labels
labels = to_categorical(labels, num_classes=len(categories))

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Check if split was successful
print(f"Training set size: {len(X_train)}")
print(f"Testing set size: {len(X_test)}")


Loaded 1930 images.
Training set size: 1544
Testing set size: 386


# Simple CNN Model

In [3]:
def simple_cnn():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(len(categories), activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model_simple = simple_cnn()
model_simple.summary()
model_simple.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


NameError: name 'Sequential' is not defined

# Deeper CNN Model

In [6]:
def deeper_cnn():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(256, activation='relu'),
        Dense(len(categories), activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model_deeper = deeper_cnn()
model_deeper.summary()
model_deeper.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 63, 63, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 30, 30, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 14, 14, 128)      

# Data Augmmented CNN Model

In [ ]:
# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen.fit(X_train)

def cnn_with_augmentation():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(256, activation='relu'),
        Dense(len(categories), activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model_augmented = cnn_with_augmentation()
model_augmented.summary()
model_augmented.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=30, validation_data=(X_test, y_test))


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 63, 63, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 30, 30, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 14, 14, 128)      

In [9]:
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop

def build_model(hp):
    model = Sequential()

    # Convolutional layers
    model.add(Conv2D(hp.Int('conv_units', min_value=32, max_value=128, step=32), (3, 3), activation='relu', input_shape=(128, 128, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(hp.Float('dropout_rate', min_value=0.0, max_value=0.5, step=0.1)))

    # Additional convolutional layers
    for i in range(hp.Int('num_conv_layers', 1, 3)):
        model.add(Conv2D(hp.Int(f'conv_units_{i}', min_value=32, max_value=128, step=32), (3, 3), activation='relu'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Dropout(hp.Float(f'dropout_rate_{i}', min_value=0.0, max_value=0.5, step=0.1)))

    model.add(Flatten())

    # Fully connected layers
    for i in range(hp.Int('num_dense_layers', 1, 3)):
        model.add(Dense(hp.Int(f'dense_units_{i}', min_value=128, max_value=512, step=128), activation='relu'))
        model.add(Dropout(hp.Float(f'dropout_rate_dense_{i}', min_value=0.0, max_value=0.5, step=0.1)))

    model.add(Dense(len(categories), activation='softmax'))

    # Compile model
    model.compile(optimizer=hp.Choice('optimizer', values=['adam', 'rmsprop']),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=20,
    executions_per_trial=1,
    directory='my_dir',
    project_name='trashnet_tuning'
)

tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]


ModuleNotFoundError: No module named 'keras_tuner'

In [12]:
pip install keras_tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.7 MB/s eta 0:00:00


In [14]:
best_model = tuner.hypermodel.build(best_hps)
best_model.summary()
best_model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))


NameError: name 'best_hps' is not defined

# Models with Optimal Hyperparameters, Regularization, and Learning Rate Scheduler

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import LearningRateScheduler

# Define learning rate scheduler
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * 0.1

lr_scheduler = LearningRateScheduler(scheduler)


In [13]:
import keras_tuner as kt

def build_model(hp):
    model = Sequential()

    # Convolutional layers with L2 regularization
    model.add(Conv2D(hp.Int('conv_units', min_value=32, max_value=128, step=32), (3, 3), activation='relu',
                     input_shape=(128, 128, 3), kernel_regularizer=l2(hp.Float('l2_reg', 1e-4, 1e-2, sampling='LOG'))))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(hp.Float('dropout_rate', min_value=0.0, max_value=0.5, step=0.1)))

    # Additional convolutional layers with L2 regularization
    for i in range(hp.Int('num_conv_layers', 1, 3)):
        model.add(Conv2D(hp.Int(f'conv_units_{i}', min_value=32, max_value=128, step=32), (3, 3), activation='relu',
                         kernel_regularizer=l2(hp.Float(f'l2_reg_{i}', 1e-4, 1e-2, sampling='LOG'))))
        model.add(MaxPooling2D((2, 2)))
        model.add(Dropout(hp.Float(f'dropout_rate_{i}', min_value=0.0, max_value=0.5, step=0.1)))

    model.add(Flatten())

    # Fully connected layers with L2 regularization
    for i in range(hp.Int('num_dense_layers', 1, 3)):
        model.add(Dense(hp.Int(f'dense_units_{i}', min_value=128, max_value=512, step=128), activation='relu',
                        kernel_regularizer=l2(hp.Float(f'l2_reg_dense_{i}', 1e-4, 1e-2, sampling='LOG'))))
        model.add(Dropout(hp.Float(f'dropout_rate_dense_{i}', min_value=0.0, max_value=0.5, step=0.1)))

    model.add(Dense(len(categories), activation='softmax'))

    # Compile model
    model.compile(optimizer=hp.Choice('optimizer', values=['adam', 'rmsprop']),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=1,
    executions_per_trial=1,
    directory='my_dir',
    project_name='trashnet_tuning'
)

tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test), callbacks=[lr_scheduler])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]


Trial 1 Complete [00h 06m 26s]
val_accuracy: 0.5984455943107605

Best val_accuracy So Far: 0.5984455943107605
Total elapsed time: 00h 06m 26s


In [14]:
best_model = tuner.hypermodel.build(best_hps)
best_model.summary()
history_best = best_model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), callbacks=[lr_scheduler])


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 63, 63, 32)        0         
 g2D)                                                            
                                                                 
 dropout_5 (Dropout)         (None, 63, 63, 32)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 61, 61, 32)        9248      
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 30, 30, 32)        0         
 g2D)                                                            
                                                                 
 dropout_6 (Dropout)         (None, 30, 30, 32)       

# Save and Load the models

In [2]:
# Save the models
model_simple.save('model_simple.h5')
model_deeper.save('model_deeper.h5')
model_augmented.save('model_augmented.h5')
best_model.save('model_optimized.h5')


NameError: name 'model_simple' is not defined

In [1]:
from tensorflow.keras.models import load_model

# Load the models
model_simple = load_model('model_simple.h5')
model_deeper = load_model('model_deeper.h5')
model_augmented = load_model('model_augmented.h5')
model_optimized = load_model('model_optimized.h5')


OSError: No file or directory found at model_simple.h5

# Comparison & Evaluation

In [17]:
# Function to evaluate and get metrics
def evaluate_model(model, X_test, y_test, model_name):
    # Get predictions
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true = np.argmax(y_test, axis=1)

    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred_classes)
    f1 = f1_score(y_true, y_pred_classes, average='weighted')
    precision = precision_score(y_true, y_pred_classes, average='weighted')
    recall = recall_score(y_true, y_pred_classes, average='weighted')

    # Calculate specificity for each class
    cm = confusion_matrix(y_true, y_pred_classes)
    specificity = np.mean([cm[i,i]/(cm[i,i] + np.sum(cm[i,:])-cm[i,i]) for i in range(cm.shape[0])])

    # Classification report
    class_report = classification_report(y_true, y_pred_classes, target_names=categories)

    # AUC-ROC
    y_test_bin = to_categorical(y_true, num_classes=len(categories))
    auc_roc = roc_auc_score(y_test_bin, y_pred, multi_class='ovr')

    print(f"{model_name} Metrics:")
    print(f"Accuracy: {accuracy}")
    print(f"F1 Score: {f1}")
    print(f"Precision: {precision}")
    print(f"Recall (Sensitivity): {recall}")
    print(f"Specificity: {specificity}")
    print(f"AUC-ROC: {auc_roc}")
    print(f"Classification Report:\n{class_report}")

    return accuracy, f1, precision, recall, specificity, auc_roc, y_true, y_pred

# Assuming your models are named model_simple, model_deeper, model_augmented, and model_optimized
metrics_simple = evaluate_model(model_simple, X_test, y_test, "Simple CNN")
metrics_deeper = evaluate_model(model_deeper, X_test, y_test, "Deeper CNN")
metrics_augmented = evaluate_model(model_augmented, X_test, y_test, "Data Augmented CNN")
metrics_optimized = evaluate_model(model_optimized, X_test, y_test, "Optimized CNN")


13/13 [==============================] - 2s 119ms/step


ValueError: Number of classes, 5, does not match size of target_names, 6. Try specifying the labels parameter

In [ ]:
# Function to plot ROC curves
def plot_roc_curve(y_true, y_pred, model_name):
    fpr = {}
    tpr = {}
    roc_auc = {}

    for i in range(len(categories)):
        fpr[i], tpr[i], _ = roc_curve(y_true[:, i], y_pred[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_true.ravel(), y_pred.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

    plt.figure()
    lw = 2
    plt.plot(fpr["micro"], tpr["micro"], color='darkorange',
             lw=lw, label='Micro-average ROC curve (area = {0:0.2f})'.format(roc_auc["micro"]))

    for i in range(len(categories)):
        plt.plot(fpr[i], tpr[i], lw=lw, label='ROC curve of class {0} (area = {1:0.2f})'.format(categories[i], roc_auc[i]))

    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'Receiver Operating Characteristic for {model_name}')
    plt.legend(loc="lower right")
    plt.show()

# Plot ROC curves for each model
plot_roc_curve(metrics_simple[6], metrics_simple[7], "Simple CNN")
plot_roc_curve(metrics_deeper[6], metrics_deeper[7], "Deeper CNN")
plot_roc_curve(metrics_augmented[6], metrics_augmented[7], "Data Augmented CNN")
plot_roc_curve(metrics_optimized[6], metrics_optimized[7], "Optimized CNN")


In [ ]:
import pandas as pd

# Compile metrics into a DataFrame
metrics_df = pd.DataFrame({
    "Model": ["Simple CNN", "Deeper CNN", "Data Augmented CNN", "Optimized CNN"],
    "Accuracy": [metrics_simple[0], metrics_deeper[0], metrics_augmented[0], metrics_optimized[0]],
    "F1 Score": [metrics_simple[1], metrics_deeper[1], metrics_augmented[1], metrics_optimized[1]],
    "Precision": [metrics_simple[2], metrics_deeper[2], metrics_augmented[2], metrics_optimized[2]],
    "Recall (Sensitivity)": [metrics_simple[3], metrics_deeper[3], metrics_augmented[3], metrics_optimized[3]],
    "Specificity": [metrics_simple[4], metrics_deeper[4], metrics_augmented[4], metrics_optimized[4]],
    "AUC-ROC": [metrics_simple[5], metrics_deeper[5], metrics_augmented[5], metrics_optimized[5]]
})

print(metrics_df)
